In [36]:
import scipy.io
import numpy.matlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.linalg import inv
import time
from mpl_toolkits.mplot3d import Axes3D
import line_profiler

In [2]:
%load_ext line_profiler
%load_ext cython

In [32]:
from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [38]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
#We need to define the macro CYTHON_TRACE=1 (cf. http://docs.cython.org/src/reference/compilation.html)

#cython: boundscheck=False
#cython: wraparound=False
import numpy as np
cimport numpy as np
import math

def rot(double f):
    return np.array([[math.cos(f), -math.sin(f)],
                     [math.sin(f), math.cos(f)]])

def cpd_R(a,b,g):
    R1=np.eye(3);
    R2=np.eye(3);
    R3=np.eye(3);
    
    R1[0:2:,0:2]=rot(a);
    R2[::2,::2]=rot(b);
    R3[1:,1:]=rot(g);
    
    R=np.dot(np.dot(R1,R2),R3)
    return R

def cpd_p(np.ndarray[double, ndim=2] X, np.ndarray[double, ndim=2] Y, double sigma2, double w):
    cdef np.ndarray G = np.zeros_like(X, dtype=np.float32)
    #cdef np.ndarray[double, ndim=2] G1, P, P1, Px, Pt1
    cdef int N, D, M
    
    N=X.shape[0]
    D=X.shape[1]
    M=Y.shape[0]    
    G=X[:,np.newaxis,:]-Y
    G=G*G
    G=np.sum(G,2)
    G=np.exp(-1.0/(2*sigma2)*G)
    G1=np.sum(G,1)
    temp2=(G1+(2*math.pi*sigma2)**(D/2)*w/(1-w)*(M/N)).reshape([N,1])
    P=(G/temp2).T
    P1=(np.sum(P,1)).reshape([M,1])
    PX=np.dot(P,X)
    Pt1=(np.sum(np.transpose(P),1)).reshape([N,1])
    return P1,Pt1,PX

def solve_problem(np.ndarray[double, ndim=2] X, np.ndarray[double, ndim=2] Y):
    #cdef np.ndarray[double, ndim=2] R3d, T
    cdef int N, D, M, max_it, iterations, L
    cdef float sigma2, sigma2_init, tol, ntol, eps
    
    #R3d=cpd_R(np.random.rand(1),np.random.rand(1),np.random.rand(1))
    R3d=np.array([
            [0.782526980767448,-0.620169604901219,-0.0551469448623274],
            [0.601199798501436,0.775670888777332,-0.192076741395768],
            [0.161896036556843,0.117150900380880,0.979829240167456]
        ])    

    X=1*np.dot(X,np.transpose(R3d))+1
    N=X.shape[0]
    D=X.shape[1]
    M=Y.shape[0]    
    sigma2=(M*np.trace(np.dot(np.transpose(X),X)) +
            N*np.trace(np.dot(np.transpose(Y),Y)) -
            2*np.dot(sum(X),np.transpose(sum(Y))))/(M*N*D)
    sigma2_init=sigma2
    tol=np.power(10.0,-5)
    max_it=150
    iterations=0
    ntol=tol+10
    L=1
    eps=np.spacing(1)
    T=Y    
    
    while (iterations<max_it) and (sigma2>np.power(10.0,-8)):
        P1,Pt1,PX=cpd_p(X,T,sigma2,0.0)
        #precompute
        Np=np.sum(Pt1)
        mu_x=np.dot(np.transpose(X),Pt1)/Np
        mu_y=np.dot(np.transpose(Y),P1)/Np

        A=np.dot(np.transpose(PX),Y)-Np*(np.dot(mu_x,np.transpose(mu_y)))
        U,S,V = np.linalg.svd(A)
        S=np.diag(S)
        C=np.eye(D)
        C[-1,-1]=np.linalg.det(np.dot(U,V))
        R=np.dot(U,np.dot(C,V))

        sigma2save=sigma2
        s=(np.trace(np.dot(S,C))/(np.sum(np.sum(Y*Y*np.matlib.repmat(P1,1,D)))-Np*
                np.dot(np.transpose(mu_y),mu_y)))

        sigma22=(np.abs(np.sum(np.sum(X*X*np.matlib.repmat(Pt1,1,D)))-Np*
                np.dot(np.transpose(mu_x),mu_x)-s*np.trace(np.dot(S,C)))/(Np*D))
        sigma2=sigma22[0][0]
        t=mu_x-np.dot(s*R,mu_y)
        T=np.dot(s*Y,np.transpose(R))+np.matlib.repmat(np.transpose(t),M,1)
        iterations+=1
    return T

<!DOCTYPE html>
<!-- Generated by Cython 0.23.4 -->
 
 
 
 Cython: _cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd.pyx 
 
 
 
 
 Generated by Cython 0.23.4 
 
 Yellow lines hint at Python interaction. 
 Click on a line that starts with a " + " to see the C code that Cython generated for it.
 
 + 01 : #We need to define the macro CYTHON_TRACE=1 (cf. http://docs.cython.org/src/reference/compilation.html) 
 __Pyx_TraceLine (1,0, {__pyx_filename = __pyx_f[0]; __pyx_lineno = 1; __pyx_clineno = __LINE__; goto __pyx_L1_error;} )
 __pyx_t_1 = PyDict_New (); if (unlikely(!__pyx_t_1)) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 1; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_GOTREF (__pyx_t_1);
 if ( PyDict_SetItem (__pyx_d, __pyx_n_s_test, __pyx_t_1) < 0) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 1; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_DECREF (__pyx_t_1); __pyx_t_1 = 0;
   02 : 
   03 : #cython: boundscheck=False 
   04 : #cython: wraparound=False 
 + 05 : import numpy as np 
 __Pyx_TraceLine (5,0, {__pyx_filename = __pyx_f[0]; __pyx_lineno = 5; __pyx_clineno = __LINE__; goto __pyx_L1_error;} )
 __pyx_t_1 = __Pyx_Import (__pyx_n_s_numpy, 0, -1); if (unlikely(!__pyx_t_1)) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 5; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_GOTREF (__pyx_t_1);
 if ( PyDict_SetItem (__pyx_d, __pyx_n_s_np, __pyx_t_1) < 0) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 5; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_DECREF (__pyx_t_1); __pyx_t_1 = 0;
   06 : cimport numpy as np 
 + 07 : import math 
 __Pyx_TraceLine (7,0, {__pyx_filename = __pyx_f[0]; __pyx_lineno = 7; __pyx_clineno = __LINE__; goto __pyx_L1_error;} )
 __pyx_t_1 = __Pyx_Import (__pyx_n_s_math, 0, -1); if (unlikely(!__pyx_t_1)) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 7; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_GOTREF (__pyx_t_1);
 if ( PyDict_SetItem (__pyx_d, __pyx_n_s_math, __pyx_t_1) < 0) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 7; __pyx_clineno = __LINE__; goto __pyx_L1_error;} 
 __Pyx_DECREF (__pyx_t_1); __pyx_t_1 = 0;
   08 : 
 + 09 : def rot ( double f ): 
 /* Python wrapper */
static PyObject *__pyx_pw_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_1rot(PyObject *__pyx_self, PyObject *__pyx_arg_f); /*proto*/
static PyMethodDef __pyx_mdef_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_1rot = {"rot", (PyCFunction)__pyx_pw_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_1rot, METH_O, 0};
static PyObject *__pyx_pw_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_1rot(PyObject *__pyx_self, PyObject *__pyx_arg_f) {
 double __pyx_v_f;
 PyObject *__pyx_r = 0;
 __Pyx_RefNannyDeclarations 
 __Pyx_RefNannySetupContext ("rot (wrapper)", 0);
 assert(__pyx_arg_f); {
 __pyx_v_f = __pyx_ PyFloat_AsDouble (__pyx_arg_f); if (unlikely((__pyx_v_f == (double)-1) && PyErr_Occurred())) {__pyx_filename = __pyx_f[0]; __pyx_lineno = 9; __pyx_clineno = __LINE__; goto __pyx_L3_error;} 
 }
 goto __pyx_L4_argument_unpacking_done;
 __pyx_L3_error:;
 __Pyx_AddTraceback ("_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd.rot", __pyx_clineno, __pyx_lineno, __pyx_filename);
 __Pyx_RefNannyFinishContext ();
 return NULL;
 __pyx_L4_argument_unpacking_done:;
 __pyx_r = __pyx_pf_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_rot(__pyx_self, ((double)__pyx_v_f));
 int __pyx_lineno = 0;
 const char *__pyx_filename = NULL;
 int __pyx_clineno = 0;

 /* function exit code */
 __Pyx_RefNannyFinishContext ();
 return __pyx_r;
}

static PyObject *__pyx_pf_46_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd_rot(CYTHON_UNUSED PyObject *__pyx_self, double __pyx_v_f) {
 PyObject *__pyx_r = NULL;
 __Pyx_TraceDeclarations 
 __Pyx_TraceFrameInit (__pyx_codeobj_)
 __Pyx_RefNannyDeclarations 
 __Pyx_RefNannySetupContext ("rot", 0);
 __Pyx_TraceCall ("rot", __pyx_f[0], 9, 0, {__pyx_filename = __pyx_f[0]; __pyx_lineno = 9; __pyx_clineno = __LINE__; goto __pyx_L1_error;} );
/* … */
 /* function exit code */
 __pyx_L1_error:;

In [39]:
# Numba versions
import numba

def rot(f):
    return np.array([[math.cos(f), -math.sin(f)],
                     [math.sin(f), math.cos(f)]])

def cpd_R(a,b,g):
    R1=np.eye(3);
    R2=np.eye(3);
    R3=np.eye(3);
    
    R1[0:2:,0:2]=rot(a);
    R2[::2,::2]=rot(b);
    R3[1:,1:]=rot(g);
    
    R=np.dot(np.dot(R1,R2),R3)
    return R

@numba.jit
def cpd_p_numba(X, Y, sigma2, w):        
    N,D=X.shape
    M,D=Y.shape    
    G=X[:,np.newaxis,:]-Y
    G=G*G
    G=np.sum(G,2)
    G=np.exp(-1.0/(2*sigma2)*G)
    G1=np.sum(G,1)
    temp2=(G1+(2*math.pi*sigma2)**(D/2)*w/(1-w)*(M/N)).reshape([N,1])
    P=(G/temp2).T
    P1=(np.sum(P,1)).reshape([M,1])
    PX=np.dot(P,X)
    Pt1=(np.sum(np.transpose(P),1)).reshape([N,1])
    return P1,Pt1,PX

@numba.jit
def solve_problem_numba(X, Y):
    #R3d=cpd_R(np.random.rand(1),np.random.rand(1),np.random.rand(1))
    R3d=np.array([
            [0.782526980767448,-0.620169604901219,-0.0551469448623274],
            [0.601199798501436,0.775670888777332,-0.192076741395768],
            [0.161896036556843,0.117150900380880,0.979829240167456]
        ])

    X=1*np.dot(X,np.transpose(R3d))+1
    N,D=X.shape
    M,D=Y.shape
    sigma2=(M*np.trace(np.dot(np.transpose(X),X)) +
            N*np.trace(np.dot(np.transpose(Y),Y)) -
            2*np.dot(sum(X),np.transpose(sum(Y))))/(M*N*D)
    sigma2_init=sigma2
    tol=np.power(10.0,-5)
    max_it=150
    iterations=0
    ntol=tol+10
    L=1
    eps=np.spacing(1)
    T=Y    
    
    while (iterations<max_it) and (sigma2>np.power(10.0,-8)):
        P1,Pt1,PX=cpd_p_numba(X,T,sigma2,0.0)
        #precompute
        Np=np.sum(Pt1)
        mu_x=np.dot(np.transpose(X),Pt1)/Np
        mu_y=np.dot(np.transpose(Y),P1)/Np

        A=np.dot(np.transpose(PX),Y)-Np*(np.dot(mu_x,np.transpose(mu_y)))
        U,S,V = np.linalg.svd(A)
        S=np.diag(S)
        C=np.eye(D)
        C[-1,-1]=np.linalg.det(np.dot(U,V))
        R=np.dot(U,np.dot(C,V))

        sigma2save=sigma2
        s=(np.trace(np.dot(S,C))/(np.sum(np.sum(Y*Y*np.matlib.repmat(P1,1,D)))-Np*
                np.dot(np.transpose(mu_y),mu_y)))

        sigma22=(np.abs(np.sum(np.sum(X*X*np.matlib.repmat(Pt1,1,D)))-Np*
                np.dot(np.transpose(mu_x),mu_x)-s*np.trace(np.dot(S,C)))/(Np*D))
        sigma2=sigma22[0][0]
        t=mu_x-np.dot(s*R,mu_y)
        T=np.dot(s*Y,np.transpose(R))+np.matlib.repmat(np.transpose(t),M,1)
        iterations+=1
    return T

In [16]:
#load data cpd_data2D_fish.mat
fish=scipy.io.loadmat(r'data\bun_zipper_res3.mat')
# initialization
X=fish['X']
Y=X

In [40]:
profile = line_profiler.LineProfiler(solve_problem, cpd_p)
profile.runcall(solve_problem, X, Y)
profile.print_stats()

Timer unit: 3.11008e-07 s

Total time: 14.3018 s
File: C:\Users\Ivo\.ipython\cython\_cython_magic_a41bfbc7a2eefa7f7b9c5c63bdc42ddd.pyx
Function: cpd_p at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def cpd_p(np.ndarray[double, ndim=2] X, np.ndarray[double, ndim=2] Y, double sigma2, double w):
    26        49         4091     83.5      0.0      cdef np.ndarray G = np.zeros_like(X, dtype=np.float32)
    27                                               #cdef np.ndarray[double, ndim=2] G1, P, P1, Px, Pt1
    28                                               cdef int N, D, M
    29                                               
    30        49          130      2.7      0.0      N=X.shape[0]
    31        49           74      1.5      0.0      D=X.shape[1]
    32        49           77      1.6      0.0      M=Y.shape[0]    
    33        49     12514123 255390.3     27.2      G=X[:,np.newaxis,:]-Y
    34        4

In [41]:
%lprun -f cpd_p -f solve_problem now=time.time();T = solve_problem(X, Y);print(time.time()-now)

14.308235168457031


In [42]:
%lprun -f cpd_p -f solve_problem_numba now=time.time();T = solve_problem_numba(X, Y);print(time.time()-now)

21.41986608505249


In [20]:
ax1=Axes3D(plt.figure(1))
ax1.plot(X[:,0],X[:,1],X[:,2],'yo')
ax1.plot(Y[:,0],Y[:,1],Y[:,2],'r+')
ax2=Axes3D(plt.figure(2))
ax2.plot(X[:,0],X[:,1],X[:,2],'yo')
ax2.plot(T[:,0],T[:,1],T[:,2],'r+')
plt.show()

C:\Users\Ivo\Anaconda3\envs\cpd\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
